In [159]:
import pandas as pd
import matplotlib as plt
from scipy.stats import t, linregress, f
import numpy as np
from sklearn import linear_model


In [160]:
cigs = pd.read_csv('https://www1.aucegypt.edu/faculty/hadi/RABE5/Data5/P088.txt', sep='\t')
cigs.to_csv('cigs.csv', index=False)


In [161]:
cigs = pd.read_csv('cigs.csv')

In [162]:
cigs['ones'] = 1

In [163]:
cigs.head()

,State,Age,HS,Income,Black,Female,Price,Sales,ones
0,AL,27.0,41.3,2948.0,26.2,51.7,42.7,89.8,1
1,AK,22.9,66.7,4644.0,3.0,45.7,41.8,121.3,1
2,AZ,26.3,58.1,3665.0,3.0,50.8,38.5,115.2,1
3,AR,29.1,39.9,2878.0,18.3,51.5,38.8,100.3,1
4,CA,28.1,62.6,4493.0,7.0,50.8,39.7,123.0,1


In [164]:
cigs.columns

Index(['State', 'Age', 'HS', 'Income', 'Black', 'Female', 'Price', 'Sales',
       'ones'],
      dtype='object')

In [165]:
X = cigs[['Age', 'HS', 'Income', 'Black', 'Female', 'Price']].to_numpy()

In [166]:
Y = cigs['Sales'].to_numpy()

In [167]:
reg = linear_model.LinearRegression()
reg.fit(X, Y)

LinearRegression()

In [168]:
reg.intercept_

103.3448457253431

Here we will use $\hat{y} = x^T\hat{\beta}$

In [189]:
X = cigs[['ones', 'Age', 'HS', 'Income', 'Black', 'Female', 'Price']].to_numpy()
b = np.expand_dims(np.concatenate(([reg.intercept_], reg.coef_)), axis=1)


and we will use:  
$F_{n} = {1\over \hat{\sigma}^2} {(Gb-c)^T(G(X^TX)^{-1}G^T))^{-1}(Gb-c) \over k} \sim F_{k, n-p}$  
under the assumption $ G\beta = c $ for $ G \in \mathbb{R}^{k \times p}$ with $ Rank(G) = k $  
and we will test by $ \varphi_{\alpha}(F_{n}) = 1\{F_{n}\ > q_{1-{\alpha}}\} $  
and $ p\_val = P(F_{k, n-p} > F_{n}) $

in 1 we have:  
 $ G = \left(\begin{matrix} 0&0&0&0&0&1&0 \end{matrix}\right)$  
 $ c = 0$  

in 2 we have:  
 $ G = \left(\begin{matrix} 0&0&0&0&0&1&0\\0&0&1&0&0&0&0 \end{matrix}\right) $  
 $ c = \left(\begin{matrix} 0 \\ 0 \end{matrix}\right) $

1:


In [170]:
n = X.shape[0]
p = X.shape[1]

In [186]:
b

array([[ 1.03344846e+02],
       [ 4.52045242e+00],
       [-6.15860528e-02],
       [ 1.89464530e-02],
       [ 3.57535168e-01],
       [-1.05285886e+00],
       [-3.25491843e+00]])

In [193]:
G = np.array([0, 0, 0, 0, 1, 0, 0], ndmin=2)
k = 1
sigma_sq = np.sum((Y -X@b) ** 2)/(n - p)
F = (1/sigma_sq * (G@b).T @ np.linalg.inv(G@np.linalg.inv(X.T@X)@G.T) @ (G@b) /k)[0, 0]
p_val = 1 - f.cdf(k, n-p, F)
p_val

0.9855559463967764

we can't reject that there is no connection.

2:

In [172]:
G = np.array([[0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0]])
k = 2
sigma_sq = np.sum((Y -X@b) ** 2)/(n - p)
F = (1/sigma_sq * (G@b).T@ (np.linalg.inv(G@np.linalg.inv(X.T@X)@G.T))@(G@b) /k)[0, 0]
p_val = 1 - f.cdf(k, n-p, F)
p_val

0.99900468693632

we can't reject that there is no connection.

3:

Here we will use:  
$ {b_{j} - c \over \sqrt{\hat{\sigma}^2e_{j}^T(X^TX)^{-1}e_{j}}} \sim t_{n-p} $  
unser the assumption $ \beta_{j} = c $

In [173]:
confidence = 0.95
alpha = 1 - confidence
j = 3 # index for income
sigma_sq = np.sum((Y -X@b) ** 2)/(n - p)
center = b[j][0]
se = np.sqrt(sigma_sq * np.linalg.inv(X.T@X)[j, j])
t_val = t.ppf(1 - alpha/2, n - p)
length = se * t_val
print([center - length, center + length])

[-0.1861036748948762, 0.22399658080085225]


4:

we will use a model assuming $b_{income} = 0$, so we will just ignore that column

In [175]:
X_1 = cigs[['Age', 'HS', 'Black', 'Female', 'Price']].to_numpy()
reg_1 = linear_model.LinearRegression()
reg_1.fit(X_1, Y)

LinearRegression()

In [178]:
var_explained = ((Y - reg_1.predict(X_1)) ** 2).sum()
total_var = ((Y - Y.mean()) ** 2).sum()

In [182]:
percentage_explained = var_explained / total_var * 100
print(percentage_explained, '%')

73.224738308099 %
